In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [ ]:
I_screws = cv2.imread('details.png', cv2.IMREAD_GRAYSCALE)
I_screws = cv2.medianBlur(I_screws, 3)

plt.imshow(I_screws, 'gray')
plt.axis('off')
plt.show()


In [ ]:
structuring_element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))

fer_top = cv2.morphologyEx(I_screws, cv2.MORPH_TOPHAT, structuring_element)
fer_bla = cv2.morphologyEx(I_screws, cv2.MORPH_BLACKHAT, structuring_element)

plt.imshow(fer_top, 'gray', vmin=0, vmax=256)
plt.axis('off')
plt.show()

plt.imshow(fer_bla, 'gray', vmin=0, vmax=256)
plt.axis('off')
plt.show()


In [ ]:
image_bin_pod = np.where(I_screws < 210, 0, 1)
image_bin_pod = image_bin_pod.astype('uint8')

plt.imshow(image_bin_pod, 'gray')
plt.axis('off')
plt.show()

In [ ]:
image_bin = np.where(fer_bla < 50, 0, 1)
image_bin = image_bin.astype('uint8')

plt.imshow(image_bin, 'gray')
plt.axis('off')
plt.show()

In [ ]:
suma = image_bin + image_bin_pod
suma = suma.astype('uint8')

plt.imshow(suma, 'gray')
plt.axis('off')
plt.show()

In [ ]:
suma_bin = np.where(suma > 0, 1, 0)
suma_bin = suma_bin.astype('uint8')
suma_bin = cv2.medianBlur(suma_bin, 3)
img_dilate = cv2.dilate(suma_bin, np.ones((3, 3)))

plt.imshow(img_dilate, 'gray')
plt.axis('off')
plt.show()

In [ ]:
labels, I_indices, stats, centroids = cv2.connectedComponentsWithStats(img_dilate)
plt.imshow(I_indices, 'gray')
plt.axis('off')
print(centroids[0, 0])

In [ ]:
ccl_vis = I_indices.copy()
s = 0
for label_num in range(2, labels + 1):
    cv2.putText(ccl_vis, str(label_num - 1), (int(centroids[label_num - 1, 0]), int(centroids[label_num - 1, 1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (20, 0, 0), 2)

plt.imshow(ccl_vis, 'gray')
plt.axis('off')

In [ ]:
moments_real = []
hu_moments_real = []
images_real = []

for i in range(1, labels):
    print("Object number: {}".format(i))
    I = (I_indices == i) * 1
    I = I.astype('uint8')
    images_real.append(I)

    plt.imshow(I, 'gray')
    plt.show()
    moments_real.append(cv2.moments(I))
    hu_moments_real.append(cv2.HuMoments(moments_real[i - 1]))
    print(moments_real[i - 1]['m00'])
    print(hu_moments_real[i - 1][0][0])
    print(hu_moments_real[i - 1][1][0])
    print(hu_moments_real[i - 1][2][0])

In [ ]:
A, B = I_indices.shape
empty_real = np.zeros((A, B))

for i in range(len(images_real)):
    if(moments_real[i]['m00'] > 319 and moments_real[i]['m00'] < 860 and hu_moments_real[i][0][0] > 0.19 and hu_moments_real[i][1][0] > 0.04):
        empty_real += images_real[i]

plt.imshow(empty_real, 'gray')
plt.show()

In [ ]:
A, B = I_indices.shape
empty_nut = np.zeros((A, B))

for i in range(len(images_real)):
    if((moments_real[i]['m00'] > 302 and moments_real[i]['m00'] < 355 and hu_moments_real[i][0][0] > 0.17 and hu_moments_real[i][0][0] < 0.196)
       or ((moments_real[i]['m00'] > 819 and moments_real[i]['m00'] < 910 and hu_moments_real[i][0][0] > 0.17 and hu_moments_real[i][0][0] < 0.20))):
        empty_nut += images_real[i]

plt.imshow(empty_nut, 'gray')
plt.show()

In [ ]:
A, B = I_indices.shape
empty_washer = np.zeros((A, B))

for i in range(len(images_real)):
    if((moments_real[i]['m00'] > 282 and moments_real[i]['m00'] < 380 and hu_moments_real[i][0][0] > 0.19 and hu_moments_real[i][0][0] < 0.26 and hu_moments_real[i][2][0] > 5.25*10**(-5))
       or ((moments_real[i]['m00'] > 886 and moments_real[i]['m00'] < 972 and hu_moments_real[i][0][0] > 0.16 and hu_moments_real[i][0][0] < 0.18))):
        empty_washer += images_real[i]

plt.imshow(empty_washer, 'gray')
plt.show()